In [185]:
!pip install psycopg2

In [192]:
import psycopg2
import pandas as pd
import psycopg2.extras

In [193]:
#creating database
conn=psycopg2.connect(host="database-1.cggevlgjyvlj.us-east-2.rds.amazonaws.com", user="postgres", password="postgres", database="recipe")

In [194]:

# Cursor is used to communicate with database
mycursor=conn.cursor()

# It will autocommit canges in database for the session
conn.set_session(autocommit=True)

query1="CREATE TABLE dish(dish_id INT PRIMARY KEY, cuisine_id INT NOT NULL)"
query2="CREATE TABLE cuisine(cuisine_id INT PRIMARY KEY, name VARCHAR NOT NULL)"
query3="CREATE TABLE recipe(recipe_id INT PRIMARY KEY,dish_id INT NOT NULL, ing_id INT NOT NULL)"
query4="CREATE TABLE ingredients(ing_id INT PRIMARY KEY, name VARCHAR NOT NULL)"
query=[query1,query2,query3,query4]

In [195]:
data=pd.read_json('train.json')

In [196]:
# For ingredients-- making a list of ingredients
ing_list=[]
for i in data['ingredients']:
    for j in i:
        ing_list.append(j)
ing_list=list(set(ing_list))

# For making a tuple-list of type of cuisine
cuisine_list=[]
for i in data['cuisine']:
    cuisine_list.append(i)
cuisine_list=list(set(cuisine_list))

In [197]:
cuisine_list[0:5]  #example

['irish', 'japanese', 'indian', 'mexican', 'moroccan']

In [199]:
# Inserting ingredients into the database table
ing_listf=[]
for i in ing_list:
    ing_listf.append((ing_list.index(i)+1000,i))
insert_query1 = 'INSERT INTO ingredients (ing_id,name) VALUES %s'
psycopg2.extras.execute_values(mycursor, insert_query1, ing_listf, template=None, page_size=10000)

In [200]:
# Inserting cuisine into the database table
cuisine_listf=[]
for i in cuisine_list:
    cuisine_listf.append((cuisine_list.index(i)+10,i))
insert_query2 = 'INSERT INTO cuisine (cuisine_id,name) VALUES %s'
psycopg2.extras.execute_values(mycursor, insert_query2, cuisine_listf, template=None, page_size=10000)

In [201]:
# Tuple list for table dish
dish_listf=[]
for (i,j) in zip(data["id"],data['cuisine']):
    dish_listf.append((i,cuisine_list.index(j)+10))
insert_query3 = 'INSERT INTO dish (dish_id,cuisine_id) VALUES %s'
psycopg2.extras.execute_values(mycursor, insert_query3, dish_listf, template=None, page_size=10000)

In [202]:
# Tuple list for recipe table
recipe_listf=[]
recipe_id=110000
for (i,j) in zip(data['id'],data['ingredients']):
    for k in j:
        recipe_listf.append((recipe_id,i,ing_list.index(k)))
        recipe_id+=1

In [203]:
len(recipe_listf)

428275

In [204]:
recipe_listf[0:5]

[(110000, 10259, 1918),
 (110001, 10259, 4049),
 (110002, 10259, 4969),
 (110003, 10259, 6107),
 (110004, 10259, 547)]